In [ ]:
%load_ext autoreload
%autoreload 2

# fad_embed

> Generate embeddings from audio files

In [ ]:
#| default_exp fad_embed

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import argparse

In [ ]:
#|export 
def embed(embed_model, real_path, fake_path, chunk_size, sr):
    print(embed_model, real_path, fake_path, chunk_size, sr)

In [ ]:
#| export
def main(): 
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('embed_model', help='chioce of embedding model')
    parser.add_argument('real_path', help='Path of files of real audio', default='real/')
    parser.add_argument('fake_path', help='Path of files of fake audio', default='fake/')
    parser.add_argument('--chunk_size', type=int, default=24000, help='Length of chunks (in audio samples) to embed')
    parser.add_argument('--sr', type=int, default=48000, help='sample rate (will resample inputs at this rate)')
    args = parser.parse_args()
    embed( args.embed_model, args.real_path, args.fake_path, args.chunk_size, args.sr)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()